# Descripción del proyecto

## Objetivo

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

In [1]:
#Se importan las librerías que se estarán utilizando para el proyecto

import pandas as pd
from sklearn import set_config
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

set_config(print_changed_only=False)

In [2]:
#Se carga el dataset con el que se estará desarrollando el modelo

df = pd.read_csv('/datasets/users_behavior.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
#Se examina la información del dataset, para conocer la información que se tiene, y de que manera se encuentra la información
#para saber como trabajarla

df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
#separar en 3 la data train_test_split
#entrena con los datos de entrenamiento
#nuevas caracter
#se hacen pruebas con los datos de entrenamiento, validacion y prueba con el accuracy_score-la metrica para calificar al modelo
#para checar la exactitud del modelo

In [5]:
df.loc[df['is_ultra'] == 1, 'plan'] = 1
df.loc[df['is_ultra'] == 0, 'plan'] = 0

In [6]:
#Se segmenta la data en 3 conjuntos: entrenamiento, validación y prueba

df_train, df_valid = train_test_split(df, test_size = 0.20, random_state = 12345)
df_valid, df_test = train_test_split(df_valid, test_size = 0.50, random_state = 12345)

In [7]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(2571, 6)
(321, 6)
(322, 6)


In [8]:
features_train = df_train.drop(['is_ultra', 'plan'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra', 'plan'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra', 'plan'], axis=1)
target_test = df_test['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(2571, 4)
(2571,)
(321, 4)
(321,)
(322, 4)
(322,)


El tamaño de los conjuntos se seleccionó asignando un porcentaje más amplio al conjunto de entrenamiento, y asignando un porcentaje más pequeño pero de igual tamaño tanto para el conjunto de validación como para el conjunto de prueba, para que con esto de como resultado una proporción de datos fuente 3:1:1.

## Calidad de diferentes modelos

A continuación se va a realizar pruebas con diferentes modelos, para revisar la calidad de estos y los resultados que arroja

### Árbol de decisión

In [9]:
model = DecisionTreeClassifier()
model.fit(features_train, target_train)
print(model)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, random_state=None,
                       splitter='best')


In [10]:
for depth in range(1, 9):
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
        model.fit(features_train, target_train)
    
        predictions_valid = model.predict(features_valid)

        print("max_depth =", depth, ": ", end='')
        print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7414330218068536
max_depth = 2 : 0.7819314641744548
max_depth = 3 : 0.7881619937694704
max_depth = 4 : 0.7881619937694704
max_depth = 5 : 0.794392523364486
max_depth = 6 : 0.7975077881619937
max_depth = 7 : 0.7912772585669782
max_depth = 8 : 0.7850467289719626


Al realizar la prueba con el modelo de arbol de decisión, se asignó en primera instancia hiperparámetros del 1 al 6, en segunda instancia se asignaron hiperparámetros del 1 al 9, para revisar los resultados que arrojaba conforme los hiperparámetros fueran aumentando, lo que nos arrojó fue que, conforme incrementaba el max_depth, iba decreciendo el resultado de nuevo.

Aquí nos arroja que el mejor resultado es la profundidad 6 dando un accuracy_score de .0797577, que dicho accuracy fue el mejor resultados de todas las pruebas y modelos que se hicieron, por lo que nos quedaríamos con este modelo el cual nos arrojó mayor exactitud.

### Bosque aleatorio

In [11]:
best_score = 0
best_est = 0
for est in range(1, 11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
    model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 7): 0.7912772585669782


Con el modelo de bosque aleatorio, podemos observar que su exactitud también es bastante aceptable, ya que su accuracy es de las más altas y que tiene más exactitud, sin embargo, no fue lo suficientemente alta como en el caso del modelo de arbol de decisión, que obtuvo mejores resultados y exactitud.

### Regresión logística

In [12]:
model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features_train,target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
score_test = model.score(features_test, target_test) # calcula la puntuación de accuracy en el conjunto de prueba

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)
print("Accuracy del modelo de regresión logística en el conjunto de prueba:", score_test)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7016725009723843
Accuracy del modelo de regresión logística en el conjunto de validación: 0.67601246105919
Accuracy del modelo de regresión logística en el conjunto de prueba: 0.7298136645962733


En el caso del modelo de regresión logística, nos arroja resultados bastante bajos, resultados que no llegan al umbral de exactitud que se pide en el proyecto, por lo que este modelo no es el ideal para nuestro proyecto

In [13]:
predictions_valid = model.predict(features_valid)
result = mean_squared_error(target_valid, predictions_valid) ** 0.5
print(result)

0.5691990328003114


## Pruebas de cordura

In [14]:
#Prueba de cordura con resultado de "0"

predictions = pd.Series(0, index = target_train.index)
print(accuracy_score(target_train, predictions))

predictions2 = pd.Series(0, index = target_valid.index)
print(accuracy_score(target_valid, predictions2))

predictions3 = pd.Series(0, index = target_test.index)
print(accuracy_score(target_test, predictions3))

0.6931155192532089
0.6791277258566978
0.7111801242236024


In [15]:
#Prueba de cordura con resultado de "1"

predictions = pd.Series(1, index = target_train.index)
print(accuracy_score(target_train, predictions))

predictions2 = pd.Series(1, index = target_valid.index)
print(accuracy_score(target_valid, predictions2))

predictions3 = pd.Series(1, index = target_test.index)
print(accuracy_score(target_test, predictions3))

0.30688448074679114
0.32087227414330216
0.2888198757763975


Al realizar las pruebas de cordura con dos opciones de resultados en los que el modelo no estaba entrenado, tanto que todos dieran "0", como otra prueba en la que todos dieran "1", para revisar los resultados arrojados y verificar la funcionalidad del modelo, lo cual nos muestra que los resultados fueron realmente bajos, lo que nos muestra que el modelo funciona como se esperaba.

## Conclusión

Como conclusión podemos ver que al realizar pruebas en los diferentes modelos de entrenamiento nos pudimos dar cuenta cual es la mejor opción, y los resultados fueron bastante similares al umbral que se tenía como objetivo, sin embargo, es bueno verificar cual es la mejor opción de modelo, para obtener los resultados deseados, así como también depende del objetivo que se desee alcanzar, puede variar el modelo, en este caso, para este proyecto, pudimos verificar que el modelo más ideal fue el Árbol de decisión que nos arrojó un accuracy con el mejor resultado y con más exactitud.

También vemos la importancia de seleccionar los hiperparámetros correctos, ya que en esta ocasión se hizo una prueba dos veces, aplicando dos rangos de hiperparámetros diferentes, para ver los resultados arrojados, y seleccionar el que mejor accuracy tenga.

Por otro lado, y por último no se puede dejar atrás las pruebas de cordura, que son my importantes para verificar la correcta funcionalidad del modelo y así estar aún más seguros de los resultados y de seleccionar el modelo correcto para llegar al objetivo que se desea.